<a href="https://colab.research.google.com/github/sdaigo/playground-transformers/blob/main/multiple_sequences.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers --quiet

     |████████████████████████████████| 4.0 MB 5.2 MB/s 
     |████████████████████████████████| 77 kB 4.2 MB/s 
     |████████████████████████████████| 6.6 MB 9.4 MB/s 
     |████████████████████████████████| 596 kB 44.7 MB/s 
     |████████████████████████████████| 880 kB 48.1 MB/s 


In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

Downloading:   0%|          | 0.00/255M [00:00<?, ?B/s]

In [5]:
import torch

sequence = "I've been waiting for a HuggingFace course my whole life."

tokens = tokenizer.tokenize(sequence)
ids = tokenizer.convert_tokens_to_ids(tokens)

input_ids = torch.tensor(ids)

In [6]:
# should be failed
model(input_ids)

IndexError: ignored

In [7]:
input_ids

tensor([ 1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,  2607,
         2026,  2878,  2166,  1012])

In [9]:
input_ids = torch.tensor([ids]) # add new dimension

output = model(input_ids)
output

SequenceClassifierOutput([('logits',
                           tensor([[-2.7276,  2.8789]], grad_fn=<AddmmBackward0>))])

In [10]:
input_ids

tensor([[ 1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,  2607,
          2026,  2878,  2166,  1012]])

In [11]:
output.logits

tensor([[-2.7276,  2.8789]], grad_fn=<AddmmBackward0>)

## Padding

複数の文章をバッチ処理しようとすると、文章の長さが異なる場合に処理ができない。

In [12]:
seq1_ids = [[200, 200, 200]]
seq2_ids = [[200, 200]]

batched_ids = [
  [200, 200, 200],
  [200, 200, tokenizer.pad_token_id]
]

In [13]:
tokenizer.pad_token_id

0

In [14]:
print(model(torch.tensor(seq1_ids)).logits)
print(model(torch.tensor(seq2_ids)).logits)
print(model(torch.tensor(batched_ids)).logits)

tensor([[ 1.5694, -1.3895]], grad_fn=<AddmmBackward0>)
tensor([[ 0.5803, -0.4125]], grad_fn=<AddmmBackward0>)
tensor([[ 1.5694, -1.3895],
        [ 1.3374, -1.2163]], grad_fn=<AddmmBackward0>)


`batched_ids` のテンソルの2行目は `seq2_ids` と同じ値になりそうだが、異なる値になっている。

Transformer モデルは各トークンをコンテキスト化する Attention Layer があるため、パディング用のトークンを無視しなければならない。

## Attention mask

Attension mask は`input_ids`テンソルと全く同じ形状のテンソルであり、0と1で埋められる。1は対応するトークンにアテンションすべきことを示し、0は対応するトークンにアテンションすべきでない（すなわち、モデルのアテンション層によって無視されるべき）ことを示す。

In [16]:
batched_ids = [
  [200, 200, 200],
  [200, 200, tokenizer.pad_token_id]
]

attention_mask = [
  [1, 1, 1],
  [1, 1, 0]  # 0 は無視すべきトークン
]

outputs = model(torch.tensor(batched_ids), attention_mask=torch.tensor(attention_mask))

outputs.logits

tensor([[ 1.5694, -1.3895],
        [ 0.5803, -0.4125]], grad_fn=<AddmmBackward0>)